In [1]:
import torch
import numpy as np
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt

from apex import amp
from utils.logger import create_logger
from datasets.build_AS import build_dataloader_no_ddp
from trainers import vificlip_P2_merged_model_AS_20Oct23

/home/asinha13/anaconda3/envs/vclip/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
/home/asinha13/projects/CLIP4ADL/vifi_hyperformer/CLIP_ADL_AS/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


In [2]:
if torch.cuda.is_available():
    torch.cuda.set_device(5)
else:
    print("No GPU")

In [3]:
from utils.config import get_config, _C

config = _C
# config.merge_from_file('./configs/base2novel/finetuning_base2novel/ntu/16_16_vifi_clip_base_train_55_5.yaml')
config.merge_from_file('./configs/base2novel/finetuning_base2novel/ntu/16_16_vifi_clip_novel_eval_55_5.yaml')

logger = create_logger(output_dir=config.OUTPUT, dist_rank=0, name="Test")

train_data, test_data, train_loader, test_loader = build_dataloader_no_ddp(logger='', config=config)
class_names = [class_name for i, class_name in test_data.classes]

In [4]:
merged_model = vificlip_P2_merged_model_AS_20Oct23.returnCLIP(config,
                            logger=logger,
                            class_names=class_names,
                            args='hyperformer')
                            # args=args) # pass args to indicate that we want to load the hyperformer model

merged_model = merged_model.cuda()

merged_model = amp.initialize(models=merged_model, optimizers=None, opt_level=config.TRAIN.OPT_LEVEL)
merged_model = merged_model.eval()

returnclip hyperformer
[2024-01-10 09:16:16 Test](vificlip_P2_merged_model_AS_20Oct23.py 316): INFO Loading CLIP (backbone: ViT-B/16)
[2024-01-10 09:16:18 Test](vificlip_P2_merged_model_AS_20Oct23.py 319): INFO Building ViFi-CLIP CLIP
vificlip hyperformer
vificlip - before load_model hyperformer
Hyperformer_Model(
  (data_bn): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (l1): TCN_ViT_unit(
    (vit1): unit_vit(
      (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
      (attn): MHSA(
        (kv): Conv2d(3, 432, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (q): Conv2d(3, 216, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Conv2d(216, 216, kernel_size=(1, 1), stride=(1, 1), groups=6)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (skip_proj): Conv2d(3, 216, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (pe

/home/asinha13/anaconda3/envs/vclip/lib/python3.7/site-packages/apex/__init__.py:68: DeprecatedFeatureWarning: apex.amp is deprecated and will be removed by the end of February 2023. Use [PyTorch AMP](https://pytorch.org/docs/stable/amp.html)
  warnings.warn(msg, DeprecatedFeatureWarning)


In [5]:
p2_pose_txt_path = './work_dirs/zero_shot/ntu_CS_KD_vcp0_posetextp1_base_zs_55_5_AS_12Nov23_v1/ckpt_epoch_29.pth'

In [6]:
checkpoint = torch.load(p2_pose_txt_path, map_location='cpu')
load_state_dict = checkpoint['model']

# now remove the unwanted keys:
if "module.prompt_learner.token_prefix" in load_state_dict:
    del load_state_dict["module.prompt_learner.token_prefix"]

if "module.prompt_learner.token_suffix" in load_state_dict:
    del load_state_dict["module.prompt_learner.token_suffix"]

if "module.prompt_learner.complete_text_embeddings" in load_state_dict:
    del load_state_dict["module.prompt_learner.complete_text_embeddings"]

new_state_dict = {}
for key in load_state_dict:
    new_key = key.replace('module.', '')
    new_state_dict[new_key] = load_state_dict[key]

mk, uk = merged_model.load_state_dict(new_state_dict, strict=False)
mk, uk

(['prompt_learner.complete_text_embeddings'], [])

In [9]:
num_samples, num_correct = 0, 0

all_pose_text_logits_78 = None
all_image_text_logits = None
accuracy_meter = []
for i, batch in enumerate(test_loader):
    if i%(len(test_loader)//10) ==0:
        print(f'{i}/{len(test_loader)}')
        print('label: ', batch['label'])

    _image = batch["imgs"]
    b, tn, c, h, w = _image.size()
    t = config.DATA.NUM_FRAMES
    n = tn // t
    _image = _image.view(b, n, t, c, h, w)
    hyperformer_data = batch["hformer_data"].cuda(non_blocking=True)
    label_id = batch["label"]
    label_id = label_id.reshape(-1)

    tot_similarity = [torch.zeros((b, config.DATA.NUM_CLASSES)).cuda() for _ in range(3)]
    for i in range(n):

        image = _image[:, i, :, :, :, :]
        label_id = label_id.cuda(non_blocking=True)
        image_input = image.cuda(non_blocking=True)

        image_features, text_features, pose_embd = merged_model(image_input, hyperformer_data)
        # text_features, pose_embd = model(image_input, hyperformer_data)

        image_text_logits = image_features @ text_features.t()
        text_pose_logits = pose_embd @ text_features.t()
        img_text_pose_logits = image_text_logits + text_pose_logits

        # logits_list = [image_text_logits, text_pose_logits, img_text_pose_logits]
        logits_list = [text_pose_logits]

        for k, logits in enumerate(logits_list):
            similarity = logits.view(b, -1).softmax(dim=-1)
            tot_similarity[k] += similarity
        
        modality_names = ['text_pose']
        
        for k in range(len(modality_names)):
            values_1, indices_1 = tot_similarity[k].topk(1, dim=-1)
            values_5, indices_5 = tot_similarity[k].topk(5, dim=-1)
            acc1, acc5 = 0, 0

            conf_matrix = torch.zeros((config.DATA.NUM_CLASSES, config.DATA.NUM_CLASSES)).cuda()
            for i in range(b):
                conf_matrix[label_id[i].item(), indices_1[i].item()] += 1
                if indices_1[i] == label_id[i]:
                    acc1 += 1
                if label_id[i] in indices_5[i]:
                    acc5 += 1

            
            accuracy_meter.append(acc1 / b)
            num_samples += b
            num_correct += acc1

average_accuracy = sum(accuracy_meter) / len(accuracy_meter)




0/2350
label:  tensor([[0],
        [3]])


IndexError: tuple index out of range

In [ ]:
average_accuracy

In [ ]:
num_correct, num_samples, num_correct/num_samples

In [7]:
num_samples, num_correct = 0, 0

all_pose_text_logits_78 = None
all_image_text_logits = None
for i, batch in enumerate(test_loader):
    if i%(len(test_loader)//10) ==0:
        print(f'{i}/{len(test_loader)}')
        print('label: ', batch['label'])

    with torch.no_grad():
        image_features, text_features, pose_embd = merged_model(batch['imgs'].cuda(), batch['hformer_data'].cuda())
        # all_image_text_logits, text_pose_logits = merged_model_2(batch['imgs'].cuda(), batch['hformer_data'].cuda())

        image_text_logits = image_features @ text_features.t()
        text_pose_logits = pose_embd @ text_features.t()

        logits = text_pose_logits.detach().cpu() # 2x60
        # logits = (all_image_text_logits + text_pose_logits).detach().cpu() # 2x60
        
        if all_pose_text_logits_78 is None and all_image_text_logits is None:
            all_pose_text_logits_78 = text_pose_logits
            all_image_text_logits = image_text_logits
        else:
            all_pose_text_logits_78 = torch.cat([all_pose_text_logits_78, text_pose_logits], dim=0)
            all_image_text_logits = torch.cat([all_image_text_logits, image_text_logits], dim=0)

        # img_text_logits_P0.append(logits)
        num_samples += logits.shape[0]
        num_correct += (logits.argmax(dim=1)==batch['label'][:, 0]).sum()

        # print(logits.argmax(dim=1), batch['label'][:, 0], f'{num_correct / num_samples}')

num_correct, num_samples, num_correct / num_samples

0/2350
label:  tensor([[0],
        [3]])
235/2350
label:  tensor([[0],
        [3]])
470/2350
label:  tensor([[0],
        [1]])
705/2350
label:  tensor([[4],
        [4]])
940/2350
label:  tensor([[0],
        [0]])
1175/2350
label:  tensor([[1],
        [1]])
1410/2350
label:  tensor([[2],
        [0]])
1645/2350
label:  tensor([[4],
        [1]])
1880/2350
label:  tensor([[1],
        [3]])
2115/2350
label:  tensor([[0],
        [4]])


(tensor(2567), 4700, tensor(0.5462))